In [8]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import csv
import os
import copy
import itertools

In [9]:
def select_mode(key):
    number = -1

    # if 48 <= key <= 57:  # 0 ~ 9
    #     number = key - 48
    # if key == 110:  # n
    #     mode = 0
    if key == 107:  # k
        mode = 1
    else:
        mode = 0
    return  mode

In [10]:
def logging_csv(number, mode, landmark_list): 
    # if mode == 0:
    #     pass
    if mode == 1 and (0 <= number <= 9):
        csv_path = 'new_keypoint.csv'
        
        # Check if the file exists, if not, create it
        if not os.path.exists(csv_path):
            with open(csv_path, 'w', newline="") as f:
                writer = csv.writer(f)
                # The file will be created but no header is written, just ready to append data

        # Append the data to the CSV file
        with open(csv_path, 'a', newline="") as f:
            writer = csv.writer(f)
            writer.writerow([number, *landmark_list])
    return


In [11]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point


In [12]:
def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list


In [13]:
def store_labels_to_new_file(csv_path, label_csv_path):
    """After collecting all the data in 'new_keypoint.csv', extract labels and store them in 'keypoint_classifier_label_new.csv'."""
    # Check if 'new_keypoint.csv' exists
    if os.path.exists(csv_path):
        # Read the data from 'new_keypoint.csv'
        df = pd.read_csv(csv_path, header=None)
        
        # If the dataframe is not empty, process the data
        if not df.empty:
            # Assuming the first column contains the labels and subsequent columns contain landmark data
            labels = df[0].values
            labels = np.unique(labels)

            # Prepare the data to be written to 'keypoint_classifier_label_new.csv'
            label_data = []
            for label in labels:# Receive the label once (no repeated labels)
                label_data.append(label)
            
            # Write to 'keypoint_classifier_label_new.csv'
            label_df = pd.DataFrame(label_data)
            label_df.to_csv(label_csv_path, index=False, header=False)
            # print(f"Labels and landmarks have been stored in '{label_csv_path}'.")

    else:
        print(f"Error: The file '{csv_path}' does not exist.")


In [14]:
# Initialize video capture
cap = cv2.VideoCapture(0)

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Unable to access the camera.")
    exit()

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,  # Whether to process each frame as a static image
    max_num_hands=1,  # Number of hands to detect
    min_detection_confidence=0.7,  # Minimum confidence value for detecting hands
    min_tracking_confidence=0.5  # Minimum confidence value for hand tracking
)




mode = 0
recCount = 0
csv_path = 'new_keypoint.csv'

# Process video frames
while True:
    if recCount == 500: 
        recCount = 0
        mode = 0

    if cv2.waitKey(10) & 0xFF == ord('k'):
        mode = 1
        
        # Check if the CSV file exists
        if os.path.exists(csv_path) and os.path.getsize(csv_path) == 0:# Check if file exists but does not have column to parse:
            # If the file exists but is empty, delete the file in local storage
            os.remove(csv_path)
        elif os.path.exists(csv_path):
            # If the file exists, read it into the DataFrame
            df = pd.read_csv(csv_path, header=None)
        else:
            # If the file doesn't exist, create an empty DataFrame with appropriate columns
            # Assuming your CSV has a structure with columns like: [label, x1, y1, x2, y2, ..., xn, yn]
            columns = [0] + [i for i in range(21)] + [j for j in range(21)]  # Example for 21 landmarks
            df = pd.DataFrame(columns=columns)
            # Optionally, save the empty DataFrame to the CSV file
            df.to_csv(csv_path, index=False)
        
    if cv2.waitKey(10) & 0xFF == ord('q'): break

    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting.")
        break

    frame = cv2.flip(frame, 1)  # Mirror the frame
    debug_frame = copy.deepcopy(frame)

    # Convert the frame to RGB (MediaPipe requires RGB)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform hand detection
    results = hands.process(frame_rgb)

    # If hands are detected, draw landmarks on the frame
    if results.multi_hand_landmarks:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):

            # Landmark calculation
            Landmark_list = calc_landmark_list(debug_frame, hand_landmarks)

            # Conversion to relative coordinates / normalized coordinates
            pre_processed_landmark_list = pre_process_landmark(Landmark_list)

            if mode == 1 and recCount < 500:
                # Create a new number for the current entry (e.g., based on the max label in the CSV)
                number = df[0].max() + 1 if not df.empty else 0
                recCount += 1
                cv2.putText(frame, 'hand record activated', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3)
                print(f"Mode: {mode}, Observation: {recCount}, Label: {number}")
                # Write to the dataset file
                logging_csv(number, mode, pre_processed_landmark_list)
                # Store labels in 'keypoint_classifier_label_new.csv'
                store_labels_to_new_file('new_keypoint.csv', 'keypoint_classifier_label_new.csv')



            # Draw the landmarks
            mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Display instructions
    cv2.putText(frame, 'Press q to exit ', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3)
    cv2.imshow('Frame', frame)

# Release the capture object and close windows
cap.release()
cv2.destroyAllWindows()


c:\Users\dangq\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\dangq\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\dangq\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabas

Mode: 1, Observation: 1, Label: 0
Mode: 1, Observation: 2, Label: 0
Mode: 1, Observation: 3, Label: 0


c:\Users\dangq\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\dangq\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\dangq\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabas

Mode: 1, Observation: 4, Label: 0
Mode: 1, Observation: 5, Label: 0
Mode: 1, Observation: 6, Label: 0


c:\Users\dangq\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\dangq\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\dangq\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabas

...

Mode: 1, Observation: 495, Label: 5
Mode: 1, Observation: 496, Label: 5
Mode: 1, Observation: 497, Label: 5


c:\Users\dangq\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\dangq\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Mode: 1, Observation: 498, Label: 5
Mode: 1, Observation: 499, Label: 5


c:\Users\dangq\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\dangq\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Mode: 1, Observation: 500, Label: 5
